In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd 'drive/My Drive/PCPPN'
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Mounted at /content/drive
/content/drive/My Drive/PCPPN


device(type='cpu')

In [ ]:
import os
import shutil

import torch
import torch.utils.data
# import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import argparse
import re

from helpers import makedir
import model
import push
import prune
import train_and_test as tnt
import save
from log import create_logger
from preprocess import mean, std, preprocess_input_function

In [ ]:
from settings import prototype_shape, num_classes
print(prototype_shape)
print(num_classes)

(16, 128, 1, 1)
4


## Check the number of images

In [ ]:
## Ok

epoches_accuracy_saved = [i for i in range(1000) if i % 10 == 0] # show the epches that the accuracy logs to be saved
# book keeping namings and code
from settings import base_architecture, img_size, prototype_shape, num_classes, \
                     prototype_activation_function, add_on_layers_type, experiment_run

base_architecture_type = re.match('^[a-z]*', base_architecture).group(0)

model_dir = './saved_models/' + base_architecture + '/' + experiment_run + '/'
makedir(model_dir)

# shutil.copy(src=os.path.join(os.getcwd(), __file__), dst=model_dir)  # I think this line should not be implemented (__file__ doesn't exist!)
shutil.copy(src=os.path.join(os.getcwd(), 'settings.py'), dst=model_dir)
shutil.copy(src=os.path.join(os.getcwd(), base_architecture_type + '_features.py'), dst=model_dir)
shutil.copy(src=os.path.join(os.getcwd(), 'model.py'), dst=model_dir)
shutil.copy(src=os.path.join(os.getcwd(), 'train_and_test.py'), dst=model_dir)

log, logclose = create_logger(log_filename=os.path.join(model_dir, 'train.log'))
img_dir = os.path.join(model_dir, 'img')
makedir(img_dir) # Directory for saving prototypes
weight_matrix_filename = 'outputL_weights'
prototype_img_filename_prefix = 'prototype-img'
prototype_self_act_filename_prefix = 'prototype-self-act'
proto_bound_boxes_filename_prefix = 'bb'

# load the data
from settings import train_dir, test_dir, train_push_dir, \
                     train_batch_size, test_batch_size, train_push_batch_size

normalize = transforms.Normalize(mean=mean,
                                 std=std)


# push set
train_push_dataset = datasets.ImageFolder(
    train_push_dir,
    transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
    ]))
train_push_loader = torch.utils.data.DataLoader(
    train_push_dataset, batch_size=1, shuffle=False,
    num_workers=4, pin_memory=False)

# test set
test_dataset = datasets.ImageFolder(
    test_dir,
    transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
        normalize,
    ]))
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=False,
    num_workers=4, pin_memory=False)

# we should look into distributed sampler more carefully at torch.utils.data.distributed.DistributedSampler(train_dataset)
#log('training set size: {0}'.format(len(train_loader.dataset)))
log('push set size: {0}'.format(len(train_push_loader.dataset)))
log('test set size: {0}'.format(len(test_loader.dataset)))


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


push set size: 995
test set size: 605


In [ ]:
# Check the data distribution over in different classes

from tqdm.auto import tqdm

cnt = torch.zeros(40)
Label = []
for im, label in tqdm(train_push_loader):
    Label.append(label)
    cnt[label] += 1

print(cnt)
print( torch.min(cnt[:num_classes]) )
print( torch.max(cnt) )

  0%|          | 0/995 [00:00<?, ?it/s]

tensor([128., 143., 355., 369.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.])
tensor(128.)
tensor(369.)


# Balancing(expand dataset to desired size data per class by duplicating)



In [ ]:
trg_size = 600  # This is set so that 1200 = trg_size*num_classes/2

import glob
from settings import train_push_dir, data_path
target_dir = data_path + 'train_push_balanced'
if not os.path.exists(target_dir):
   os.mkdir(target_dir)

for l in range (0, num_classes):
    f_name = str(l)
    if l < 10:
       f_name = '0' + str(l)
    print('class:', l)
    src_dir = train_push_dir + f_name
    dst_dir = target_dir + '/' + f_name
    if not os.path.exists(dst_dir):
       os.mkdir(dst_dir)

    cnt_copy = 0
    while cnt_copy < trg_size:
      for filename in tqdm( glob.iglob(src_dir+ '/**/*.png', recursive = True), desc = "Reading" ):
          img_name = filename.split('/')[9]
          #print(img_name)
          src_file = filename
          dst_file = dst_dir + '/' + str(cnt_copy) + '_' + img_name # + str(cnt_copy) + '_'
          shutil.copy(src_file, dst_file)
          cnt_copy += 1
          print(cnt_copy)
          if cnt_copy >= trg_size:
             break




class: 0


Reading: 0it [00:00, ?it/s]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128


Reading: 0it [00:00, ?it/s]

129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256


...

356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
class: 3


Reading: 0it [00:00, ?it/s]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


Reading: 0it [00:00, ?it/s]

370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600


## Augmentation

In [ ]:
pip install Augmentor

In [ ]:
import Augmentor
import os
import time

from settings import data_path
d = data_path + 'train_push_balanced/'

d2 = 'train_augmented'

In [ ]:
for i in range (0, 1): # (0 , k) means that the data will be augmented k*4 times
  # rotation
  p = Augmentor.Pipeline(source_directory = d, output_directory = d2)
  p.rotate(probability=1, max_left_rotation=15, max_right_rotation=15)
  p.flip_left_right(probability=0.5)
  p.process()
  time.sleep(7 * 60)

  #skew
  p = Augmentor.Pipeline(source_directory=d, output_directory=d2)
  p.skew(probability=1, magnitude=0.2)  # max 45 degrees
  p.flip_left_right(probability=0.5)
  p.process()
  time.sleep(7 * 60)

  # shear
  p = Augmentor.Pipeline(source_directory=d, output_directory=d2)
  p.shear(probability=1, max_shear_left=10, max_shear_right=10)
  p.flip_left_right(probability=0.5)
  p.process()
  time.sleep(7 * 60)

  #random_distortion
  p = Augmentor.Pipeline(source_directory=d, output_directory=d2)
  p.random_distortion(probability=1.0, grid_width=10, grid_height=10, magnitude=5)
  p.flip_left_right(probability=0.5)
  p.process()
  time.sleep(7 * 60)
  print(i)


Initialised with 2400 image(s) found.
Output directory set to /content/drive/My Drive/pseudo-class generation/datasets_exp_4/fold1_40x/train_push_balanced/train_augmented.

Processing <PIL.Image.Image image mode=RGB size=700x460 at 0x7B7E15167940>: 100%|██████████| 2400/2400 [07:47<00:00,  5.14 Samples/s]


Initialised with 2400 image(s) found.
Output directory set to /content/drive/My Drive/pseudo-class generation/datasets_exp_4/fold1_40x/train_push_balanced/train_augmented.

Processing <PIL.Image.Image image mode=RGB size=700x460 at 0x7B7E16E52E90>: 100%|██████████| 2400/2400 [07:22<00:00,  5.43 Samples/s]


Initialised with 2400 image(s) found.
Output directory set to /content/drive/My Drive/pseudo-class generation/datasets_exp_4/fold1_40x/train_push_balanced/train_augmented.

Processing <PIL.Image.Image image mode=RGB size=700x460 at 0x7B7E15164940>: 100%|██████████| 2400/2400 [07:36<00:00,  5.26 Samples/s]


Initialised with 2400 image(s) found.
Output directory set to /content/drive/My Drive/pseudo-class generation/datasets_exp_4/fold1_40x/train_push_balanced/train_augmented.

Processing <PIL.Image.Image image mode=RGB size=700x460 at 0x7B7E151B71C0>: 100%|██████████| 2400/2400 [07:25<00:00,  5.39 Samples/s]


0
